## Data Story (Draft)
### Terrorism and the media

This data story follows an exploratory approach. We'll analyze and visulaize two datasets to discover patterns and relationships in the data to form arguments to support the different perspectives that we'll see throughout this data story. 

We'll focus on how terrorism is escalating in western countries as well as non-western countries. Moreover, we'll try to find out if this is related to the way how the media presents these crimes to the public. As we see that the media espicially foucusses on terrorism in the west.

In [3]:
# Import packages
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [4]:
# Load global terrorism and population datasets using pandas
df_1 = pd.read_csv('globalterrorismdb_0718dist.csv', encoding = 'latin-1', low_memory = False)
df_2 = pd.read_csv('Population.csv')

In [10]:
#data preprocessing
df_1['west_or_not'] = df_1['region'].apply(lambda x: '1' if x == 1 or x == 12 or x == 8 else '0')
df_1['Event'] = 1

population = df_2.sort_values(by=['country_txt'])

filtered = df_1.filter(items=['country', 'Event', 'success', 'nkill', 'nwound'])
grouped = filtered.groupby("country").sum()
grouped = grouped.sort_values(by=['country'])

filteredwest = df_1.filter(items=['country', 'country_txt', 'west_or_not'])
groupedwest = filteredwest.drop_duplicates(subset=['country'])
groupedwest = groupedwest.sort_values(by=['country'])

totalset = groupedwest.join(grouped, how="inner", on="country")
totalset = totalset.merge(population, how='inner', on='country_txt')
totalset = totalset.drop(columns=['country', 'World', '#']) 

totalset = totalset.reset_index()
totalset = totalset.drop(columns=["index"])

west = totalset[totalset['west_or_not'] != '0']
nonwest = totalset[totalset['west_or_not'] != '1']

In [31]:
def cases_count(df_1, column_name):

    cases_count = df_1.groupby(column_name)['eventid'].count()
    
    # Create a new DataFrame from the Series object
    cases_count_df = pd.DataFrame({column_name: cases_count.index, 'Number of Cases': cases_count.values})

    # Sort the DataFrame by the Number of Cases column in descending order
    cases_count_df = cases_count_df.sort_values('Number of Cases', ascending=False)
    
    
    return cases_count_df

In [32]:
def cases_sum(df_1, col_group_by, value):

    cases_sum = df_1.groupby(col_group_by)[value].sum()
    
    # Create a new DataFrame from the Series object
    cases_sum_df = pd.DataFrame({col_group_by: cases_sum.index, value: cases_sum.values})

    # Sort the DataFrame by the Number of Cases column in descending order
    cases_sum_df = cases_sum_df.sort_values(value, ascending=False)
    
    
    return cases_sum_df


In [33]:

# Total cases
cases_by_year_df = cases_count(df_1,"iyear")

# Extended cases (incident extended more than 24 hours)
extended_cases_group_by_df = cases_sum(df_1,'iyear','extended')

# Success cases
success_cases_group_by_df = cases_sum(df_1,'iyear','success')

# Suicide cases
suicide_cases_group_by_df = cases_sum(df_1,'iyear','suicide')

# Merge columns on "iyear"
cases_by_year_df = pd.merge(cases_by_year_df, extended_cases_group_by_df, on = 'iyear', how='left').merge(success_cases_group_by_df, on = 'iyear').merge(suicide_cases_group_by_df, on = 'iyear')


In [30]:
# Create a line chart using Plotly
cases_by_year_df = cases_by_year_df.sort_values('iyear', ascending=False)

# Create Scatter traces for extended cases and success cases and suicide cases
total_cases = go.Scatter(x=cases_by_year_df['iyear'], y=cases_by_year_df['Number of Cases'], mode='lines', name='Total cases')
success_cases = go.Scatter(x=cases_by_year_df['iyear'], y=cases_by_year_df['success'], mode='lines', name='Success cases')
extended_cases = go.Scatter(x=cases_by_year_df['iyear'], y=cases_by_year_df['extended'], mode='lines', name='Extended cases')
suicide_cases = go.Scatter(x=cases_by_year_df['iyear'], y=cases_by_year_df['suicide'], mode='lines', name='Suicide cases')

# Add traces to a single Figure object
fig = go.Figure(data=[total_cases, success_cases, extended_cases, suicide_cases])
fig.update_layout(title='Cases over Time', xaxis_title='Year', yaxis_title='Cases')
fig.show()

The above plot indicates an alarming trend of increasing terrorism incidents over the years. With this growth in crime we can expect the media to be directly more involved in reporting these incidents. This takes us to the follwoing investigation which is the way in which the media represents these incidents to the public.

In [11]:
cases_count_df = cases_count(df_1, "country_txt")

# create choropleth map
fig = px.choropleth(cases_count_df, 
                    locations='country_txt',
                    locationmode='country names',
                    color='Number of Cases',
                    hover_name='country_txt',
                    projection='natural earth')

fig.show()

     country_txt  Number of Cases
84          Iraq            24636
134     Pakistan            14368
0    Afghanistan            12731
80         India            11960
35      Colombia             8306


In [15]:
fig = go.Figure()
fig.add_trace(go.Bar(name='Aantal aanslagen per land', x= west['country_txt'], y=west['Event']))
fig.add_trace(go.Bar(name='Aantal doden per land', x= west['country_txt'], y=west['nkill']))
fig.add_trace(go.Bar(name='Aantal succesvolle aanslagen per land', x= west['country_txt'], y=west['success']))

fig = fig.update_layout(
    title="Aantal (succesvolle) aanslagen en gevallen doden in westerse landen",
    autosize=True,
    annotations=[dict(xref='paper',
                    yref='paper',
                    y= -0.45,
                    showarrow=False,
                    text='Westerse landen')])

fig.show()

In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=west["country_txt"].unique(), y=west["nkill"], name="Aantal doodden", mode="lines"))
fig.add_trace(go.Scatter(x=west["country_txt"].unique(), y=west["nwound"], name="Aantal gewonden", mode="lines"))

fig.add_trace(go.Scatter(x=nonwest["country_txt"].unique(), y=nonwest["nkill"], name="Aantal doodden", mode="lines"))
fig.add_trace(go.Scatter(x=nonwest["country_txt"].unique(), y=nonwest["nwound"], name="Aantal gewonden", mode="lines"))

fig = fig.update_layout(
            updatemenus=[
                dict(
                type='buttons',
                direction='left',
                buttons=list([
                    dict(
                        label = "Westerse landen",
                        method = "update",
                        args = [{"visibale": [True, True, False, False]},
                               {"title": "Aantal doodden en gewonden westersw landen"}]
                    ),
                     dict(
                         label = "Niet westerse landen",
                         method = "update",
                         args = [{"visibale": [False, False, True, True]},
                               {"title": "Aantal doodden en gewonden niet westersw landen"}]
                     )
                ])
                )
            ])

fig.show()

In [19]:
labels = nonwest['country_txt']
values = nonwest['Event']

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='none',
                            insidetextorientation='radial'
                            )])

fig.show()

In [20]:
totalset = totalset.groupby("west_or_not").sum()
totalset['placement'] = ['Non westren country', 'Westren country']
totalset['equated'] = totalset['Event'] / totalset['Population']

In [21]:
labels = totalset['placement']
values1 = totalset['Event']
values2 = totalset['equated']

fig = go.Figure()
colors = ['red', 'green']
fig.add_trace(
    go.Pie(labels=labels, values=values1, hole=.8, textinfo='label+percent'))
fig.add_trace(
    go.Pie(labels=labels, values=values2, hole=.8, textinfo='label+percent', visible=False))

fig = fig.update_layout(
            updatemenus=[
                dict(
                type = 'buttons',
                direction = 'left',
                buttons = list([
                    dict(
                        label = "Origineel",
                        method = "update",
                        args = [{"visibale": [True, False]},
                               {"title": "Aantal aanslagen in westerse en niet westers landen"}]
                    ),
                    dict(
                        label = "Gelijkgesteld",
                        method = "update",
                        args = [{"visible": [False, True]},
                               {"title": "Aantal aanslagen in westerse en niet westerse landen (gelijkgesteld)"}]
                    )
                ])
                )
            ])

fig = fig.update_traces(marker=dict(colors=colors, line=dict(color='#000000', width=1)))
fig = fig.update_layout(title_text='Aantal aanslagen westerse en niet westerse landen', showlegend=False)

fig.show()